# RAG (Retrieval-Augmented Generation)
## Índice de Contenidos

1. [Fundamentos Teóricos de RAG](#1-fundamentos)
2. [Embeddings: El Corazón Semántico](#2-embeddings)
3. [Chunking: División Inteligente de Documentos](#3-chunking)
4. [Vector Stores: Almacenamiento y Búsqueda](#4-vectorstores)
5. [Pipeline RAG Completo](#5-pipeline)
6. [Técnicas Avanzadas de Retrieval](#6-avanzadas)
7. [Evaluación y Métricas](#7-evaluacion)
8. [Prompt Engineering para RAG](#8-prompting)
9. [Soluciones Open Source](#9-opensource)
10. [Despliegue en Producción](#10-produccion)

---

## Configuración del Entorno

Antes de comenzar, necesitamos instalar las bibliotecas necesarias. Este notebook utiliza herramientas open source que pueden ejecutarse completamente en local, sin necesidad de APIs externas de pago.

### Bibliotecas principales:

- **sentence-transformers**: Modelos de embeddings de última generación
- **chromadb**: Base de datos vectorial ligera y fácil de usar
- **faiss-cpu**: Biblioteca de Facebook para búsqueda de similitud eficiente
- **langchain**: Framework para construir aplicaciones con LLMs
- **transformers**: Biblioteca de Hugging Face para modelos de lenguaje
- **rank_bm25**: Implementación del algoritmo BM25 para búsqueda léxica

In [ ]:
# Instalación de dependencias (ejecutar solo la primera vez)
!pip install -q \
    langchain \
    langchain-community \
    langchain-huggingface \
    sentence-transformers \
    chromadb \
    faiss-cpu \
    transformers \
    torch \
    pypdf \
    rank_bm25 \
    scikit-learn \
    matplotlib \
    numpy \
    pandas

In [ ]:
# Imports principales que usaremos a lo largo del notebook
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

print("✅ Entorno configurado correctamente")

---

<a id="1-fundamentos"></a>
# 1. Fundamentos Teóricos de RAG

## 1.1 ¿Qué es RAG?

**RAG (Retrieval-Augmented Generation)** es una arquitectura que combina dos componentes fundamentales:

1. **Retrieval (Recuperación)**: Un sistema que busca y recupera información relevante de una base de conocimiento
2. **Generation (Generación)**: Un modelo de lenguaje (LLM) que genera respuestas basándose en la información recuperada

Esta combinación permite que los LLMs accedan a información actualizada, específica y verificable, superando sus limitaciones inherentes.

## 1.2 ¿Por qué necesitamos RAG?

Los Modelos de Lenguaje Grande (LLMs) como GPT, Llama o Mistral tienen limitaciones importantes:

### Problemas de los LLMs sin RAG:

| Problema | Descripción | Impacto |
|----------|-------------|----------|
| **Conocimiento desactualizado** | El modelo solo conoce información hasta su fecha de entrenamiento | No puede responder sobre eventos recientes |
| **Alucinaciones** | Genera información plausible pero falsa con total confianza | Respuestas incorrectas presentadas como hechos |
| **Sin acceso a datos privados** | No conoce documentos internos de tu organización | No puede ayudar con información específica |
| **Falta de citaciones** | No puede indicar de dónde proviene la información | Imposible verificar las respuestas |
| **Contexto limitado** | Ventana de contexto finita (4K-128K tokens) | No puede procesar grandes volúmenes de documentos |

### Cómo RAG resuelve estos problemas:

| Problema | Solución con RAG |
|----------|------------------|
| Conocimiento desactualizado | Acceso a documentos actualizados en tiempo real |
| Alucinaciones | Respuestas fundamentadas en fuentes específicas |
| Sin datos privados | Integración con repositorios documentales internos |
| Falta de citaciones | Referencias exactas a documentos y páginas |
| Contexto limitado | Recuperación selectiva de información relevante |

## 1.3 Arquitectura de un Sistema RAG

Un sistema RAG típico consta de dos fases principales:

### Fase 1: Indexación (Offline)

Esta fase se ejecuta una vez (o periódicamente) para preparar los documentos:

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│ DOCUMENTOS  │────▶│  CHUNKING   │────▶│  EMBEDDING  │────▶│VECTOR STORE │
│ (PDF, DOCX, │     │ (División   │     │ (Conversión │     │(Almacena-   │
│  TXT, HTML) │     │  en partes) │     │ a vectores) │     │ miento)     │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
```

1. **Carga de documentos**: Se leen los documentos en diversos formatos
2. **Chunking**: Se dividen en fragmentos manejables (típicamente 500-1000 tokens)
3. **Embedding**: Cada fragmento se convierte en un vector numérico que captura su significado
4. **Almacenamiento**: Los vectores se guardan en una base de datos vectorial con índices para búsqueda rápida

### Fase 2: Consulta (Online)

Esta fase se ejecuta cada vez que un usuario hace una pregunta:

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  CONSULTA   │────▶│  EMBEDDING  │────▶│  BÚSQUEDA   │
│ del usuario │     │ de consulta │     │  VECTORIAL  │
└─────────────┘     └─────────────┘     └──────┬──────┘
                                               │
                                               ▼
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  RESPUESTA  │◀────│     LLM     │◀────│  CONTEXTO   │
│ al usuario  │     │ (Generación)│     │ (Top-K docs)│
└─────────────┘     └─────────────┘     └─────────────┘
```

1. **Embedding de consulta**: La pregunta del usuario se convierte en vector
2. **Búsqueda**: Se encuentran los K fragmentos más similares semánticamente
3. **Construcción de contexto**: Se prepara un prompt con los documentos recuperados
4. **Generación**: El LLM genera una respuesta basada en el contexto
5. **Respuesta**: Se devuelve la respuesta con las fuentes citadas

## 1.4 Componentes Clave

### Modelo de Embeddings

Convierte texto en vectores numéricos de alta dimensión (típicamente 384-4096 dimensiones). Textos con significado similar producen vectores cercanos en el espacio vectorial.

**Ejemplos de modelos:**
- `BAAI/bge-m3` (Estado del arte, multilingüe)
- `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` (Ligero, multilingüe)
- `intfloat/multilingual-e5-large` (Excelente para español)

### Base de Datos Vectorial

Almacena los vectores y permite búsquedas eficientes por similitud.

**Ejemplos:**
- **ChromaDB**: Simple, ideal para prototipos
- **FAISS**: Alto rendimiento, desarrollado por Facebook
- **Qdrant**: Producción, filtros avanzados
- **Milvus**: Escalabilidad masiva

### Modelo de Lenguaje (LLM)

Genera la respuesta final basándose en el contexto recuperado.

**Modelos Open Source recomendados:**
- **Llama 3.1** (8B-70B): Propósito general, excelente rendimiento
- **Mistral/Mixtral**: Eficiente, buen razonamiento
- **Qwen 2.5**: Excelente soporte multilingüe

## 1.5 Ventajas de RAG para Investigación Científica

Para instituciones como el CSIC, RAG ofrece ventajas específicas:

1. **Soberanía de datos**: Los documentos nunca salen de la infraestructura propia
2. **Reproducibilidad**: Las respuestas son verificables y citables
3. **Actualización continua**: Se pueden añadir nuevas publicaciones sin reentrenar
4. **Cumplimiento normativo**: Compatible con RGPD al no enviar datos a terceros
5. **Personalización**: Adaptable a terminología y dominios específicos

---

<a id="2-embeddings"></a>
# 2. Embeddings: El Corazón Semántico de RAG

## 2.1 ¿Qué son los Embeddings?

Los **embeddings** son representaciones numéricas (vectores) de texto que capturan el **significado semántico**. A diferencia de métodos tradicionales como TF-IDF o bag-of-words, los embeddings:

- Capturan **relaciones semánticas** entre palabras y frases
- Representan el **significado** más allá de las palabras exactas
- Permiten **comparar similitud** entre textos de forma matemática

### Intuición Visual

Imagina un espacio de muchas dimensiones donde cada punto representa un texto:

- "El gato duerme en el sofá" y "El felino descansa en el sillón" estarían **cerca** (mismo significado)
- "La proteína se pliega" y "El gato duerme" estarían **lejos** (significados diferentes)

### Propiedades Matemáticas

Los embeddings tienen propiedades algebraicas interesantes:

```
vector("rey") - vector("hombre") + vector("mujer") ≈ vector("reina")
```

Esta propiedad demuestra que los embeddings capturan relaciones semánticas complejas.

## 2.2 Cómo Funcionan los Modelos de Embeddings

Los modelos de embeddings modernos se basan en arquitecturas **Transformer** (como BERT) entrenados con técnicas de **aprendizaje contrastivo**:

1. **Pares positivos**: Textos relacionados (pregunta-respuesta, paráfrasis) deben tener embeddings similares
2. **Pares negativos**: Textos no relacionados deben tener embeddings diferentes
3. **Entrenamiento**: El modelo aprende a maximizar la similitud de pares positivos y minimizar la de negativos

### Tipos de Modelos de Embeddings

| Tipo | Descripción | Ejemplo |
|------|-------------|----------|
| **Bi-encoder** | Codifica query y documento independientemente. Rápido para búsqueda. | Sentence-BERT, BGE |
| **Cross-encoder** | Procesa query y documento juntos. Más preciso pero más lento. | ms-marco-MiniLM |
| **Sparse embeddings** | Vectores dispersos, interpretables. | SPLADE |
| **Dense embeddings** | Vectores densos, capturan semántica profunda. | E5, BGE-M3 |

## 2.3 Métricas de Similitud

Para comparar embeddings usamos métricas de distancia o similitud:

### Similitud Coseno (más común)

Mide el ángulo entre dos vectores, ignorando su magnitud:

$$\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$$

- **Valor 1**: Vectores idénticos en dirección (muy similares)
- **Valor 0**: Vectores perpendiculares (sin relación)
- **Valor -1**: Vectores opuestos (significados opuestos)

### Distancia Euclidiana

Mide la distancia directa entre dos puntos:

$$\text{euclidean\_distance}(A, B) = \sqrt{\sum_{i=1}^{n}(A_i - B_i)^2}$$

### Producto Punto (Dot Product)

Similar al coseno pero considera la magnitud:

$$\text{dot\_product}(A, B) = \sum_{i=1}^{n} A_i \times B_i$$

**Recomendación**: Para RAG, usar **similitud coseno** o **producto punto con vectores normalizados**.

## 2.4 Práctica: Trabajando con Embeddings

Vamos a explorar cómo funcionan los embeddings en la práctica. Usaremos `sentence-transformers`, una biblioteca que facilita el uso de modelos de embeddings de última generación.

In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar un modelo de embeddings multilingüe
# Este modelo funciona bien con español y es relativamente ligero
print("Cargando modelo de embeddings...")
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Información del modelo
dimension = embedding_model.get_sentence_embedding_dimension()
print(f"✅ Modelo cargado correctamente")
print(f"📐 Dimensión de los embeddings: {dimension}")
print(f"📝 Esto significa que cada texto se convierte en un vector de {dimension} números")

### Ejemplo 1: Generando Embeddings

Veamos cómo convertir textos en vectores y explorar sus propiedades:

In [ ]:
# Textos de ejemplo sobre diferentes temas
textos = [
    "El gato duerme plácidamente en el sofá",           # Tema: animales/hogar
    "El felino descansa sobre el sillón de la sala",    # Paráfrasis del anterior
    "Las proteínas chaperonas ayudan al plegamiento",   # Tema: biología molecular
    "Hsp70 asiste en el correcto doblez de proteínas",  # Paráfrasis del anterior
    "Python es un lenguaje de programación versátil",   # Tema: programación
]

# Generar embeddings para todos los textos
embeddings = embedding_model.encode(textos)

# Explorar la estructura de los embeddings
print("Estructura de los embeddings generados:")
print(f"  - Tipo: {type(embeddings)}")
print(f"  - Shape: {embeddings.shape}")
print(f"  - Tenemos {embeddings.shape[0]} textos, cada uno con {embeddings.shape[1]} dimensiones")
print(f"\nPrimeras 10 dimensiones del primer embedding:")
print(f"  {embeddings[0][:10]}")

### Ejemplo 2: Matriz de Similitud

Ahora calculemos la similitud coseno entre todos los pares de textos. Esto nos mostrará qué textos son semánticamente similares:

In [ ]:
# Calcular la matriz de similitud coseno
similarity_matrix = cosine_similarity(embeddings)

# Crear una visualización clara
print("MATRIZ DE SIMILITUD COSENO")
print("=" * 80)
print("\nValores cercanos a 1.0 = textos muy similares")
print("Valores cercanos a 0.0 = textos no relacionados\n")

# Mostrar los textos con índices
print("Textos:")
for i, texto in enumerate(textos):
    print(f"  T{i}: {texto}")

print("\nMatriz de similitud:")
print("     ", end="")
for i in range(len(textos)):
    print(f"  T{i}  ", end="")
print()

for i in range(len(textos)):
    print(f"T{i}  ", end="")
    for j in range(len(textos)):
        sim = similarity_matrix[i][j]
        # Colorear según similitud (usando símbolos)
        if i == j:
            print(f" 1.00 ", end="")
        elif sim > 0.7:
            print(f" {sim:.2f}*", end="")
        else:
            print(f" {sim:.2f} ", end="")
    print()

print("\n* = Alta similitud (>0.7)")
print("\n💡 Observaciones:")
print("   - T0 y T1 (gato/felino) tienen alta similitud: son paráfrasis")
print("   - T2 y T3 (proteínas/Hsp70) tienen alta similitud: mismo tema científico")
print("   - T4 (Python) tiene baja similitud con todos: tema diferente")

### Ejemplo 3: Visualización en 2D

Aunque los embeddings tienen muchas dimensiones (384 en este caso), podemos usar **PCA (Análisis de Componentes Principales)** para proyectarlos a 2D y visualizar cómo se agrupan los textos similares:

In [ ]:
# Reducir a 2 dimensiones para visualización
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Configurar el gráfico
plt.figure(figsize=(12, 8))

# Colores por tema
colores = ['#e74c3c', '#c0392b',  # Rojo: gato/felino
           '#3498db', '#2980b9',  # Azul: proteínas
           '#27ae60']             # Verde: programación

# Dibujar puntos
for i, (x, y) in enumerate(embeddings_2d):
    plt.scatter(x, y, c=colores[i], s=200, alpha=0.7, edgecolors='white', linewidth=2)
    
    # Etiqueta abreviada
    etiqueta = textos[i][:25] + "..." if len(textos[i]) > 25 else textos[i]
    plt.annotate(f"T{i}: {etiqueta}", (x, y), 
                 fontsize=9, 
                 xytext=(10, 10), 
                 textcoords='offset points',
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))

# Dibujar líneas entre textos similares
plt.plot([embeddings_2d[0][0], embeddings_2d[1][0]], 
         [embeddings_2d[0][1], embeddings_2d[1][1]], 
         'r--', alpha=0.5, linewidth=2, label='Paráfrasis (gato/felino)')
plt.plot([embeddings_2d[2][0], embeddings_2d[3][0]], 
         [embeddings_2d[2][1], embeddings_2d[3][1]], 
         'b--', alpha=0.5, linewidth=2, label='Paráfrasis (proteínas)')

plt.title("Visualización de Embeddings en 2D (PCA)\nTextos similares aparecen cercanos", fontsize=14)
plt.xlabel(f"Componente Principal 1 ({pca.explained_variance_ratio_[0]*100:.1f}% varianza)")
plt.ylabel(f"Componente Principal 2 ({pca.explained_variance_ratio_[1]*100:.1f}% varianza)")
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 El gráfico muestra cómo los textos con significados similares")
print("   se agrupan en el espacio vectorial, incluso usando diferentes palabras.")

### Ejemplo 4: Búsqueda Semántica Simple

Implementemos una búsqueda semántica básica para entender cómo RAG encuentra documentos relevantes:

In [ ]:
def busqueda_semantica(consulta, documentos, modelo, top_k=3):
    """
    Realiza una búsqueda semántica simple.
    
    Args:
        consulta: La pregunta o búsqueda del usuario
        documentos: Lista de documentos donde buscar
        modelo: Modelo de embeddings
        top_k: Número de resultados a devolver
    
    Returns:
        Lista de tuplas (documento, score) ordenada por relevancia
    """
    # Generar embedding de la consulta
    embedding_consulta = modelo.encode([consulta])
    
    # Generar embeddings de los documentos
    embeddings_docs = modelo.encode(documentos)
    
    # Calcular similitud coseno
    similitudes = cosine_similarity(embedding_consulta, embeddings_docs)[0]
    
    # Ordenar por similitud descendente
    indices_ordenados = np.argsort(similitudes)[::-1][:top_k]
    
    # Devolver resultados
    resultados = [(documentos[i], similitudes[i]) for i in indices_ordenados]
    return resultados


# Base de conocimiento de ejemplo (simulando documentos científicos)
base_conocimiento = [
    "Las proteínas chaperonas son moléculas que asisten en el plegamiento correcto de otras proteínas.",
    "Hsp70 reconoce secuencias hidrofóbicas de 7-8 aminoácidos en proteínas desplegadas.",
    "El mal plegamiento de proteínas está asociado con enfermedades neurodegenerativas como Alzheimer.",
    "CRISPR-Cas9 permite editar secuencias de ADN con alta precisión.",
    "La PCR amplifica secuencias específicas de ADN para su análisis.",
    "Los ribosomas son los orgánulos encargados de sintetizar proteínas.",
    "Hsp90 es esencial para la maduración de receptores de hormonas esteroides.",
    "Las enfermedades por priones son causadas por proteínas mal plegadas infecciosas.",
]

# Realizar una búsqueda
consulta = "¿Qué proteínas ayudan a otras proteínas a plegarse correctamente?"

print(f"🔍 Consulta: \"{consulta}\"")
print("\n📄 Resultados de la búsqueda semántica:\n")

resultados = busqueda_semantica(consulta, base_conocimiento, embedding_model, top_k=3)

for i, (doc, score) in enumerate(resultados, 1):
    print(f"{i}. [Similitud: {score:.4f}]")
    print(f"   {doc}\n")

## 2.5 Modelos de Embeddings Recomendados

La elección del modelo de embeddings es crucial para el rendimiento de RAG. Aquí una comparativa detallada:

### Modelos Multilingües (Recomendados para español)

| Modelo | Dimensión | Contexto | Características | Caso de Uso |
|--------|-----------|----------|-----------------|-------------|
| **BAAI/bge-m3** | 1024 | 8192 tokens | Estado del arte, soporta dense + sparse | Producción, máxima calidad |
| **intfloat/multilingual-e5-large** | 1024 | 512 tokens | Muy buen rendimiento en español | RAG multilingüe |
| **sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2** | 384 | 128 tokens | Ligero y rápido | Prototipos, recursos limitados |

### Modelos Específicos para Español

| Modelo | Dimensión | Características |
|--------|-----------|----------------|
| **jinaai/jina-embeddings-v2-base-es** | 768 | Optimizado para español |
| **hiiamsid/sentence_similarity_spanish_es** | 768 | Entrenado específicamente en español |

### Criterios de Selección

1. **Idioma**: Priorizar modelos multilingües o específicos para español
2. **Dimensión**: Mayor dimensión = más capacidad semántica pero más memoria
3. **Contexto máximo**: Importante si los chunks son largos
4. **Velocidad**: Considerar latencia en producción
5. **Licencia**: Verificar compatibilidad con uso comercial/académico

---

<a id="3-chunking"></a>
# 3. Chunking: División Inteligente de Documentos

## 3.1 ¿Por qué es Necesario el Chunking?

El **chunking** (fragmentación) es el proceso de dividir documentos largos en fragmentos más pequeños. Es necesario por varias razones:

### Limitaciones Técnicas

1. **Límite de tokens del modelo de embeddings**: La mayoría de modelos tienen un máximo de 512-8192 tokens
2. **Ventana de contexto del LLM**: Aunque algunos LLMs aceptan 128K tokens, el rendimiento decrece con contextos muy largos
3. **Memoria y velocidad**: Procesar documentos completos es ineficiente

### Beneficios para la Calidad

1. **Precisión en la búsqueda**: Fragmentos pequeños permiten recuperar información más específica
2. **Relevancia del contexto**: El LLM recibe solo la información pertinente, no todo el documento
3. **Mejor citación**: Se puede indicar exactamente qué parte del documento se usó

## 3.2 El Dilema del Tamaño del Chunk

Elegir el tamaño correcto es un balance:

```
Chunks MUY PEQUEÑOS              Chunks MUY GRANDES
        ◄─────────────────────────────────────────►
        
✓ Alta precisión en búsqueda    ✗ Baja precisión en búsqueda
✗ Pérdida de contexto           ✓ Contexto completo
✗ Fragmentación de ideas        ✓ Ideas completas
✓ Menos tokens por consulta     ✗ Más tokens por consulta
```

### Recomendaciones Generales

| Caso de Uso | Tamaño Recomendado | Overlap |
|-------------|-------------------|----------|
| Documentos técnicos/científicos | 500-1000 tokens | 10-20% |
| FAQs y documentos cortos | 200-500 tokens | 5-10% |
| Código fuente | 100-300 tokens | 0-10% |
| Conversaciones/chat | 300-500 tokens | 20-30% |

## 3.3 Estrategias de Chunking

### 1. Fixed Size Chunking (Tamaño Fijo)

La estrategia más simple: dividir por número de caracteres o tokens.

**Ventajas:**
- Simple de implementar
- Chunks de tamaño predecible

**Desventajas:**
- Puede cortar a mitad de oración o párrafo
- No respeta la estructura del documento

### 2. Recursive Character Splitting (Recomendado)

Divide usando una jerarquía de separadores: primero intenta dividir por párrafos, luego por oraciones, luego por palabras.

**Ventajas:**
- Respeta mejor las unidades semánticas
- Balance entre simplicidad y calidad

**Desventajas:**
- Tamaños de chunks menos uniformes

### 3. Semantic Chunking

Usa embeddings para detectar cambios de tema y dividir en esos puntos.

**Ventajas:**
- Chunks coherentes temáticamente
- Ideal para documentos con múltiples temas

**Desventajas:**
- Más costoso computacionalmente
- Requiere calibración

### 4. Document Structure Chunking

Respeta la estructura del documento (títulos, secciones, párrafos).

**Ventajas:**
- Preserva la organización original
- Mantiene metadatos de sección

**Desventajas:**
- Requiere documentos bien estructurados
- Tamaños muy variables

## 3.4 El Concepto de Overlap (Solapamiento)

El **overlap** es la cantidad de texto que se repite entre chunks consecutivos. Es crucial para no perder contexto en los bordes:

```
Sin overlap:
┌────────────────┐ ┌────────────────┐ ┌────────────────┐
│    Chunk 1     │ │    Chunk 2     │ │    Chunk 3     │
└────────────────┘ └────────────────┘ └────────────────┘
                  ▲                  ▲
            Información perdida en los bordes

Con overlap del 20%:
┌────────────────────┐
│      Chunk 1       │
└──────────┬─────────┘
     ┌─────┴─────────────┐
     │      Chunk 2      │
     └──────────┬────────┘
          ┌─────┴────────────┐
          │     Chunk 3      │
          └──────────────────┘
```

El overlap asegura que la información que aparece en la frontera entre chunks no se pierda.

## 3.5 Práctica: Implementando Diferentes Estrategias de Chunking

Vamos a implementar y comparar diferentes estrategias de chunking con un documento científico de ejemplo.

In [ ]:
# Documento científico de ejemplo
documento_cientifico = """
# Proteínas Chaperonas: Guardianes del Plegamiento Celular

## Introducción

Las proteínas chaperonas representan una clase fundamental de moléculas que asisten en el 
plegamiento correcto de otras proteínas. Descubiertas inicialmente como proteínas de choque 
térmico (Heat Shock Proteins, HSPs), hoy sabemos que desempeñan funciones esenciales en 
condiciones normales y de estrés celular.

El plegamiento de proteínas es un proceso crítico: una cadena polipeptídica debe adoptar 
una estructura tridimensional específica para ser funcional. Errores en este proceso pueden 
llevar a enfermedades graves, incluyendo Alzheimer, Parkinson y otras patologías 
neurodegenerativas.

## Mecanismo de Acción

Las chaperonas funcionan reconociendo regiones hidrofóbicas que quedan expuestas durante 
el plegamiento. En una proteína correctamente plegada, estas regiones están ocultas en el 
interior; su exposición indica un estado de plegamiento incorrecto o incompleto.

El mecanismo general implica:

1. Reconocimiento de la proteína sustrato desplegada
2. Unión a través de dominios específicos de unión a sustrato
3. Ciclos de unión y liberación dependientes de ATP
4. Facilitación del plegamiento correcto o direccionamiento a degradación

## Familia Hsp70

La familia Hsp70 es una de las más conservadas evolutivamente y mejor caracterizadas. 
Sus miembros se encuentran en prácticamente todos los organismos, desde bacterias hasta 
humanos.

Características principales de Hsp70:

- Reconoce segmentos hidrofóbicos de 7-8 aminoácidos
- Requiere co-chaperonas como Hsp40 (DnaJ) para su actividad óptima
- Funciona mediante un ciclo ATPasa regulado
- Participa en múltiples procesos: plegamiento, translocación, desensamblaje de complejos

## Familia Hsp90

Hsp90 es esencial para la maduración de un conjunto específico de proteínas cliente, 
muchas de las cuales son factores de señalización celular.

Entre sus clientes se incluyen:

- Receptores de hormonas esteroides
- Quinasas de señalización
- Factores de transcripción
- Proteínas del ciclo celular

## Relevancia Clínica

Las chaperonas tienen una relevancia clínica significativa en múltiples contextos:

### Enfermedades Neurodegenerativas

El mal plegamiento y agregación de proteínas es la causa subyacente de numerosas 
enfermedades neurodegenerativas:

- Alzheimer: agregados de beta-amiloide y tau hiperfosforilada
- Parkinson: agregados de alfa-sinucleína (cuerpos de Lewy)
- Huntington: agregados de huntingtina con expansión de poliglutamina
- Enfermedades por priones: agregados de PrPSc

### Cáncer

Las células tumorales frecuentemente muestran dependencia de chaperonas, particularmente 
Hsp90. Esta "adicción a chaperonas" ha llevado al desarrollo de inhibidores de Hsp90 
como estrategia terapéutica anticancerígena.

## Conclusiones

Las proteínas chaperonas son componentes esenciales de la maquinaria celular de control 
de calidad proteica. Su estudio continúa revelando nuevos mecanismos y posibilidades 
terapéuticas para enfermedades asociadas al mal plegamiento de proteínas.
"""

print(f"📄 Documento cargado")
print(f"   - Longitud total: {len(documento_cientifico)} caracteres")
print(f"   - Palabras aproximadas: {len(documento_cientifico.split())}")

In [ ]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    MarkdownHeaderTextSplitter,
)

# ============================================================
# ESTRATEGIA 1: Character Text Splitter (División simple)
# ============================================================

print("="*70)
print("ESTRATEGIA 1: Character Text Splitter")
print("="*70)
print("\nDivide por un separador específico (ej: salto de línea)")
print("Simple pero puede cortar oraciones a mitad.\n")

char_splitter = CharacterTextSplitter(
    separator="\n",      # Dividir por saltos de línea
    chunk_size=500,      # Máximo 500 caracteres por chunk
    chunk_overlap=50,    # 50 caracteres de overlap
    length_function=len,
)

chunks_char = char_splitter.split_text(documento_cientifico)

print(f"Número de chunks generados: {len(chunks_char)}")
print(f"\nEjemplo - Chunk 1 ({len(chunks_char[0])} chars):")
print("-" * 50)
print(chunks_char[0][:300] + "..." if len(chunks_char[0]) > 300 else chunks_char[0])

In [ ]:
# ============================================================
# ESTRATEGIA 2: Recursive Character Text Splitter (RECOMENDADA)
# ============================================================

print("="*70)
print("ESTRATEGIA 2: Recursive Character Text Splitter (RECOMENDADA)")
print("="*70)
print("\nIntenta dividir jerárquicamente: párrafos → oraciones → palabras")
print("Mejor preservación de unidades semánticas.\n")

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,  # 20% de overlap
    length_function=len,
    separators=["\n\n", "\n", ". ", ", ", " ", ""]  # Jerarquía de separadores
)

chunks_recursive = recursive_splitter.split_text(documento_cientifico)

print(f"Número de chunks generados: {len(chunks_recursive)}")
print(f"\nTamaños de los primeros 5 chunks:")
for i, chunk in enumerate(chunks_recursive[:5]):
    print(f"  Chunk {i+1}: {len(chunk)} caracteres")

print(f"\nEjemplo - Chunk 3:")
print("-" * 50)
print(chunks_recursive[2])

In [ ]:
# ============================================================
# ESTRATEGIA 3: Markdown Header Text Splitter
# ============================================================

print("="*70)
print("ESTRATEGIA 3: Markdown Header Text Splitter")
print("="*70)
print("\nDivide respetando la estructura de encabezados Markdown.")
print("Preserva metadatos de sección para mejor contexto.\n")

# Definir los headers que queremos usar para dividir
headers_to_split = [
    ("#", "titulo"),
    ("##", "seccion"),
    ("###", "subseccion"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split
)

chunks_markdown = markdown_splitter.split_text(documento_cientifico)

print(f"Número de chunks generados: {len(chunks_markdown)}")
print(f"\nChunks con sus metadatos de sección:")
print("-" * 50)

for i, chunk in enumerate(chunks_markdown[:4]):
    print(f"\nChunk {i+1}:")
    print(f"  📁 Metadatos: {chunk.metadata}")
    contenido_preview = chunk.page_content[:150].replace('\n', ' ')
    print(f"  📝 Contenido: {contenido_preview}...")

## 3.6 Combinando Estrategias: Pipeline de Chunking Óptimo

En la práctica, la mejor aproximación es combinar estrategias: primero dividir por estructura del documento, luego aplicar chunking recursivo a secciones grandes.

Además, es importante **enriquecer cada chunk con metadatos** que ayuden en la recuperación y citación:

In [ ]:
def chunking_optimizado(documento, chunk_size=500, chunk_overlap=100):
    """
    Pipeline de chunking optimizado que combina estructura y tamaño.
    
    1. Primero divide por estructura (headers markdown)
    2. Luego aplica chunking recursivo a secciones grandes
    3. Preserva metadatos de origen
    """
    from langchain.schema import Document
    
    # Paso 1: Dividir por estructura
    headers = [("#", "titulo"), ("##", "seccion"), ("###", "subseccion")]
    md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers)
    chunks_estructura = md_splitter.split_text(documento)
    
    # Paso 2: Aplicar chunking recursivo a secciones grandes
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " "]
    )
    
    chunks_finales = []
    
    for chunk in chunks_estructura:
        contenido = chunk.page_content
        metadata = chunk.metadata.copy()
        
        if len(contenido) > chunk_size:
            # Sección grande: dividir más
            sub_chunks = recursive_splitter.split_text(contenido)
            for i, sub_chunk in enumerate(sub_chunks):
                new_metadata = metadata.copy()
                new_metadata['parte'] = i + 1
                new_metadata['total_partes'] = len(sub_chunks)
                chunks_finales.append(Document(
                    page_content=sub_chunk,
                    metadata=new_metadata
                ))
        else:
            # Sección pequeña: mantener como está
            metadata['parte'] = 1
            metadata['total_partes'] = 1
            chunks_finales.append(Document(
                page_content=contenido,
                metadata=metadata
            ))
    
    return chunks_finales


# Aplicar el chunking optimizado
chunks_optimizados = chunking_optimizado(documento_cientifico, chunk_size=400, chunk_overlap=80)

print("CHUNKING OPTIMIZADO: Resultados")
print("=" * 70)
print(f"\nTotal de chunks generados: {len(chunks_optimizados)}")
print(f"\nDetalle de cada chunk:")
print("-" * 70)

for i, chunk in enumerate(chunks_optimizados):
    seccion = chunk.metadata.get('seccion', 'Principal')
    parte = chunk.metadata.get('parte', 1)
    total = chunk.metadata.get('total_partes', 1)
    
    print(f"\nChunk {i+1}:")
    print(f"  📁 Sección: {seccion}")
    print(f"  📄 Parte: {parte}/{total}")
    print(f"  📏 Longitud: {len(chunk.page_content)} caracteres")
    print(f"  📝 Preview: {chunk.page_content[:80].replace(chr(10), ' ')}...")

## 3.7 Mejores Prácticas de Chunking

### Lista de Verificación para Chunking de Calidad

1. **Preservar unidades semánticas**
   - No cortar a mitad de oración
   - Preferir cortes en límites de párrafo
   - Respetar estructura del documento cuando sea posible

2. **Configurar overlap adecuado**
   - 10-20% del tamaño del chunk es lo recomendado
   - Más overlap para documentos con ideas que fluyen entre párrafos
   - Menos overlap para documentos con secciones independientes

3. **Enriquecer con metadatos**
   - Fuente del documento (archivo, URL)
   - Número de página o sección
   - Fecha de creación/actualización
   - Autor o departamento
   - Categoría o tags

4. **Limpiar el texto**
   - Eliminar headers/footers repetitivos
   - Corregir errores de OCR si aplica
   - Normalizar espacios y saltos de línea
   - Manejar tablas y figuras apropiadamente

5. **Adaptar al dominio**
   - Documentos científicos: chunks más grandes para mantener contexto
   - FAQs: chunks más pequeños, uno por pregunta
   - Código: respetar funciones/clases como unidades

### Parámetros Recomendados por Tipo de Documento

| Tipo de Documento | chunk_size | chunk_overlap | Estrategia |
|-------------------|------------|---------------|------------|
| Papers científicos | 800-1200 | 150-200 | Recursive + Structure |
| Documentación técnica | 500-800 | 100-150 | Recursive |
| Normativas/Legal | 600-1000 | 100-200 | Structure-aware |
| FAQs | 200-400 | 50 | Por pregunta |
| Código fuente | 200-500 | 50-100 | Por función/clase |

---

<a id="4-vectorstores"></a>
# 4. Vector Stores: Almacenamiento y Búsqueda Semántica

## 4.1 ¿Qué es un Vector Store?

Un **Vector Store** (o base de datos vectorial) es un sistema especializado en almacenar y buscar vectores de alta dimensión. A diferencia de las bases de datos tradicionales que buscan por coincidencia exacta, los vector stores permiten **búsqueda por similitud**: encontrar los vectores más parecidos a uno dado.

### ¿Por qué no usar una base de datos tradicional?

Las bases de datos relacionales (PostgreSQL, MySQL) o documentales (MongoDB) no están optimizadas para:

1. **Almacenar vectores de alta dimensión** (384-4096 dimensiones)
2. **Búsqueda por similitud** en tiempo sublineal
3. **Escalabilidad** con millones de vectores

Los vector stores utilizan estructuras de datos especializadas como **HNSW (Hierarchical Navigable Small World)** o **IVF (Inverted File Index)** que permiten búsquedas aproximadas muy rápidas.

## 4.2 Algoritmos de Indexación

### Búsqueda Exacta (Brute Force)

Compara el vector de consulta con **todos** los vectores almacenados.

- **Complejidad**: O(n) donde n = número de vectores
- **Ventaja**: 100% de precisión (recall perfecto)
- **Desventaja**: Muy lento para grandes colecciones
- **Uso**: Colecciones pequeñas (<10,000 vectores)

### HNSW (Hierarchical Navigable Small World)

Construye un grafo multicapa donde cada nodo está conectado a sus vecinos más cercanos.

```
Capa 3:    ○───────────────○  (pocos nodos, conexiones largas)
           │               │
Capa 2:    ○───○───────○───○  (más nodos)
           │   │       │   │
Capa 1:    ○─○─○─○───○─○─○─○  (más nodos aún)
           │ │ │ │   │ │ │ │
Capa 0:    ○○○○○○○○○○○○○○○○○  (todos los nodos)
```

- **Complejidad**: O(log n) para búsqueda
- **Ventaja**: Muy rápido, alta precisión (>95%)
- **Desventaja**: Mayor uso de memoria
- **Uso**: La mayoría de aplicaciones en producción

### IVF (Inverted File Index)

Divide el espacio vectorial en clusters (regiones). Para buscar, primero identifica los clusters relevantes y luego busca solo dentro de ellos.

- **Complejidad**: O(n/k) donde k = número de clusters visitados
- **Ventaja**: Buen balance velocidad/memoria
- **Desventaja**: Requiere entrenamiento previo
- **Uso**: Colecciones muy grandes (>1M vectores)

## 4.3 Comparativa de Soluciones

| Solución | Licencia | Tipo | Mejor Para | Características |
|----------|----------|------|------------|----------------|
| **ChromaDB** | Apache 2.0 | Embebido | Prototipos, desarrollo | Simple, Python-native, sin servidor |
| **FAISS** | MIT | Biblioteca | Alto rendimiento | Facebook, muy optimizado, bajo nivel |
| **Qdrant** | Apache 2.0 | Servidor | Producción | Filtros, REST API, escalable |
| **Milvus** | Apache 2.0 | Servidor | Enterprise | Cloud-native, escalabilidad masiva |
| **Weaviate** | BSD-3 | Servidor | Búsqueda híbrida | GraphQL, módulos ML integrados |
| **Pinecone** | Propietario | Cloud | Managed service | Sin operaciones, serverless |
| **pgvector** | PostgreSQL | Extensión | Integración SQL | Usa PostgreSQL existente |

### Recomendaciones por Caso de Uso

- **Aprendizaje/Prototipos**: ChromaDB (simple, sin configuración)
- **Proyecto pequeño/mediano**: Qdrant o ChromaDB persistente
- **Producción con alto rendimiento**: Qdrant, Milvus o FAISS
- **Ya tienes PostgreSQL**: pgvector
- **Sin recursos para operar**: Pinecone (cloud managed)

## 4.4 Práctica: Implementación con ChromaDB

ChromaDB es ideal para aprender y prototipar porque:
- Se ejecuta en memoria o con persistencia local
- No requiere servidor externo
- API simple e intuitiva
- Integración nativa con modelos de embeddings

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# ============================================================
# Crear cliente de ChromaDB
# ============================================================

# Opción 1: En memoria (se pierde al cerrar)
client = chromadb.Client()

# Opción 2: Persistente (descomentar para usar)
# client = chromadb.PersistentClient(path="./chroma_db")

print("✅ Cliente ChromaDB creado")

# ============================================================
# Configurar función de embeddings
# ============================================================

# Usar sentence-transformers (mismo modelo que usamos antes)
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

print("✅ Función de embeddings configurada")

In [ ]:
# ============================================================
# Crear una colección
# ============================================================

# Las colecciones son como "tablas" en bases de datos relacionales
# Cada colección tiene su propio índice y puede tener diferentes configuraciones

collection = client.get_or_create_collection(
    name="documentos_cientificos",
    embedding_function=embedding_fn,
    metadata={
        "hnsw:space": "cosine",  # Métrica de distancia: cosine, l2, ip
        "description": "Colección de documentos científicos sobre biología molecular"
    }
)

print(f"✅ Colección '{collection.name}' creada/obtenida")
print(f"   Documentos actuales: {collection.count()}")

In [ ]:
# ============================================================
# Añadir documentos a la colección
# ============================================================

# Documentos de ejemplo (simulando chunks de un paper científico)
documentos = [
    "Las proteínas chaperonas son moléculas esenciales que asisten en el plegamiento correcto de otras proteínas mediante ciclos de unión y liberación dependientes de ATP.",
    "La familia Hsp70 reconoce secuencias hidrofóbicas de 7-8 aminoácidos en proteínas parcialmente desplegadas o mal plegadas.",
    "Hsp90 es crucial para la maduración de receptores de hormonas esteroides, quinasas de señalización y factores de transcripción.",
    "El mal plegamiento de proteínas está asociado con enfermedades neurodegenerativas como Alzheimer, Parkinson y enfermedades por priones.",
    "CRISPR-Cas9 es una herramienta de edición genética que permite modificar secuencias de ADN con alta precisión usando un ARN guía.",
    "La reacción en cadena de la polimerasa (PCR) permite amplificar secuencias específicas de ADN millones de veces para su análisis.",
    "Los ribosomas son complejos macromoleculares responsables de la síntesis de proteínas, traduciendo el ARN mensajero a cadenas polipeptídicas.",
    "La espectroscopía de masas es una técnica analítica que permite identificar proteínas y caracterizar sus modificaciones post-traduccionales.",
    "Las co-chaperonas como Hsp40 (DnaJ) regulan la actividad ATPasa de Hsp70 y ayudan en el reconocimiento de sustratos.",
    "Los inhibidores de Hsp90 son una estrategia terapéutica prometedora en cáncer debido a la dependencia de las células tumorales en esta chaperona."
]

# Metadatos para cada documento
metadatos = [
    {"tema": "chaperonas", "subtema": "general", "fuente": "review_chaperonas.pdf", "pagina": 1},
    {"tema": "chaperonas", "subtema": "hsp70", "fuente": "review_chaperonas.pdf", "pagina": 5},
    {"tema": "chaperonas", "subtema": "hsp90", "fuente": "review_chaperonas.pdf", "pagina": 8},
    {"tema": "chaperonas", "subtema": "patologia", "fuente": "neurodegeneration.pdf", "pagina": 12},
    {"tema": "genetica", "subtema": "crispr", "fuente": "gene_editing.pdf", "pagina": 3},
    {"tema": "genetica", "subtema": "pcr", "fuente": "molecular_techniques.pdf", "pagina": 7},
    {"tema": "biologia_celular", "subtema": "ribosomas", "fuente": "cell_biology.pdf", "pagina": 45},
    {"tema": "proteomica", "subtema": "espectrometria", "fuente": "proteomics_handbook.pdf", "pagina": 23},
    {"tema": "chaperonas", "subtema": "cochaperonas", "fuente": "review_chaperonas.pdf", "pagina": 6},
    {"tema": "chaperonas", "subtema": "terapeutica", "fuente": "cancer_therapy.pdf", "pagina": 15},
]

# IDs únicos para cada documento
ids = [f"doc_{i:03d}" for i in range(len(documentos))]

# Añadir a la colección
collection.add(
    documents=documentos,
    metadatas=metadatos,
    ids=ids
)

print(f"✅ {len(documentos)} documentos añadidos a la colección")
print(f"   Total de documentos en la colección: {collection.count()}")

## 4.5 Búsqueda Semántica con ChromaDB

Ahora que tenemos documentos indexados, podemos realizar diferentes tipos de búsqueda:

In [ ]:
# ============================================================
# Búsqueda semántica básica
# ============================================================

consulta = "¿Qué moléculas ayudan a las proteínas a plegarse correctamente?"

resultados = collection.query(
    query_texts=[consulta],  # ChromaDB genera el embedding automáticamente
    n_results=3,              # Top 3 resultados
    include=["documents", "metadatas", "distances"]  # Qué incluir en resultados
)

print(f"🔍 BÚSQUEDA SEMÁNTICA BÁSICA")
print(f"="*70)
print(f"Consulta: \"{consulta}\"")
print(f"\n📄 Resultados (Top 3):\n")

for i in range(len(resultados['documents'][0])):
    doc = resultados['documents'][0][i]
    meta = resultados['metadatas'][0][i]
    dist = resultados['distances'][0][i]
    similitud = 1 - dist  # Convertir distancia a similitud
    
    print(f"{i+1}. [Similitud: {similitud:.4f}]")
    print(f"   📁 Fuente: {meta['fuente']}, página {meta['pagina']}")
    print(f"   🏷️  Tema: {meta['tema']} > {meta['subtema']}")
    print(f"   📝 {doc[:100]}...\n")

In [ ]:
# ============================================================
# Búsqueda con filtros de metadatos
# ============================================================

# Podemos filtrar por metadatos ANTES de la búsqueda semántica
# Esto es muy útil para acotar el dominio de búsqueda

consulta = "mecanismo de acción molecular"

print(f"🔍 BÚSQUEDA CON FILTROS DE METADATOS")
print(f"="*70)
print(f"Consulta: \"{consulta}\"")
print(f"Filtro: Solo documentos del tema 'chaperonas'\n")

resultados_filtrados = collection.query(
    query_texts=[consulta],
    n_results=5,
    where={"tema": "chaperonas"},  # Filtro de metadatos
    include=["documents", "metadatas", "distances"]
)

print(f"📄 Resultados (filtrados por tema='chaperonas'):\n")

for i in range(len(resultados_filtrados['documents'][0])):
    doc = resultados_filtrados['documents'][0][i]
    meta = resultados_filtrados['metadatas'][0][i]
    dist = resultados_filtrados['distances'][0][i]
    
    print(f"{i+1}. [{meta['subtema']}] {doc[:80]}...")

In [ ]:
# ============================================================
# Filtros avanzados con operadores lógicos
# ============================================================

print(f"🔍 FILTROS AVANZADOS")
print(f"="*70)

# Ejemplo 1: OR - documentos de chaperonas O genética
print("\n1️⃣ Filtro OR: tema = 'chaperonas' O tema = 'genetica'")
resultados_or = collection.query(
    query_texts=["técnicas de laboratorio"],
    n_results=4,
    where={
        "$or": [
            {"tema": "chaperonas"},
            {"tema": "genetica"}
        ]
    },
    include=["documents", "metadatas"]
)

for i, doc in enumerate(resultados_or['documents'][0]):
    tema = resultados_or['metadatas'][0][i]['tema']
    print(f"   {i+1}. [{tema}] {doc[:60]}...")

# Ejemplo 2: AND + comparación numérica
print("\n2️⃣ Filtro AND: tema = 'chaperonas' Y pagina < 10")
resultados_and = collection.query(
    query_texts=["función proteica"],
    n_results=4,
    where={
        "$and": [
            {"tema": "chaperonas"},
            {"pagina": {"$lt": 10}}  # $lt = less than
        ]
    },
    include=["documents", "metadatas"]
)

for i, doc in enumerate(resultados_and['documents'][0]):
    pagina = resultados_and['metadatas'][0][i]['pagina']
    print(f"   {i+1}. [pág. {pagina}] {doc[:60]}...")

## 4.6 Implementación con FAISS

**FAISS** (Facebook AI Similarity Search) es una biblioteca de bajo nivel que ofrece máximo rendimiento. Aunque requiere más código, es ideal cuando se necesita escala o control fino sobre el índice.

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Cargar modelo de embeddings (reusar el anterior si ya está cargado)
if 'embedding_model' not in dir():
    embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generar embeddings para los documentos
embeddings_docs = embedding_model.encode(documentos)
embeddings_docs = embeddings_docs.astype('float32')  # FAISS requiere float32

# Normalizar para usar producto punto (equivalente a coseno)
faiss.normalize_L2(embeddings_docs)

# Crear índice FAISS
dimension = embeddings_docs.shape[1]
index = faiss.IndexFlatIP(dimension)  # IP = Inner Product (coseno con normalización)

# Añadir vectores al índice
index.add(embeddings_docs)

print(f"✅ Índice FAISS creado")
print(f"   Dimensión: {dimension}")
print(f"   Vectores indexados: {index.ntotal}")
print(f"   Tipo de índice: Flat (búsqueda exacta)")

In [ ]:
# ============================================================
# Búsqueda con FAISS
# ============================================================

def buscar_faiss(consulta, index, modelo, documentos, k=3):
    """Realiza búsqueda semántica usando FAISS."""
    # Generar y normalizar embedding de consulta
    query_embedding = modelo.encode([consulta]).astype('float32')
    faiss.normalize_L2(query_embedding)
    
    # Buscar los k vectores más cercanos
    scores, indices = index.search(query_embedding, k)
    
    # Formatear resultados
    resultados = []
    for i, (idx, score) in enumerate(zip(indices[0], scores[0])):
        resultados.append({
            'documento': documentos[idx],
            'score': score,
            'indice': idx
        })
    
    return resultados


# Realizar búsqueda
consulta = "proteínas relacionadas con enfermedades cerebrales"

print(f"🔍 BÚSQUEDA CON FAISS")
print(f"="*70)
print(f"Consulta: \"{consulta}\"\n")

resultados_faiss = buscar_faiss(consulta, index, embedding_model, documentos, k=3)

for i, r in enumerate(resultados_faiss, 1):
    print(f"{i}. [Score: {r['score']:.4f}]")
    print(f"   {r['documento'][:80]}...\n")

## 4.7 Mejores Prácticas para Vector Stores

### Diseño de Colecciones

1. **Separar por dominio o tipo de documento**
   - Una colección para papers científicos
   - Otra para documentación técnica
   - Esto optimiza las búsquedas y permite configuraciones específicas

2. **Metadatos estratégicos**
   - Incluir información que permita filtrar (fecha, autor, categoría)
   - Guardar referencia a la fuente original (archivo, URL, página)
   - Considerar metadatos para control de acceso si es necesario

### Optimización de Rendimiento

| Factor | Recomendación |
|--------|---------------|
| < 10K docs | Índice Flat (exacto) |
| 10K - 1M docs | HNSW con parámetros default |
| > 1M docs | IVF + tuning de parámetros |
| Filtros frecuentes | Crear índices en campos de metadatos |

### Mantenimiento

1. **Actualizaciones incrementales**: Añadir nuevos documentos sin reindexar todo
2. **Versionado**: Mantener versiones del índice para rollback
3. **Backups**: Respaldar tanto los vectores como los metadatos
4. **Monitorización**: Tracking de latencias y calidad de resultados

---

<a id="5-pipeline"></a>
# 5. Pipeline RAG Completo

Ahora que entendemos cada componente (embeddings, chunking, vector stores), vamos a integrar todo en un **pipeline RAG funcional**.

## 5.1 Arquitectura del Pipeline

Un pipeline RAG completo tiene las siguientes etapas:

```
                    FASE DE INDEXACIÓN
┌──────────────────────────────────────────────────────────┐
│                                                          │
│  [Documentos] → [Chunking] → [Embedding] → [VectorStore] │
│                                                          │
└──────────────────────────────────────────────────────────┘

                    FASE DE CONSULTA
┌──────────────────────────────────────────────────────────┐
│                                                          │
│  [Query] → [Embedding] → [Retrieval] → [Contexto]       │
│                              │                           │
│                              ↓                           │
│  [Respuesta] ← [LLM] ← [Prompt con Contexto]            │
│                                                          │
└──────────────────────────────────────────────────────────┘
```

## 5.2 Implementación Paso a Paso

Vamos a construir el pipeline de forma modular, componente por componente.

In [ ]:
# ============================================================
# CLASE RAGPipeline: Implementación completa
# ============================================================

from typing import List, Dict, Optional
from dataclasses import dataclass
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

@dataclass
class Documento:
    """Representa un documento con su contenido y metadatos."""
    contenido: str
    metadata: Dict = None
    
    def __post_init__(self):
        if self.metadata is None:
            self.metadata = {}


@dataclass
class ResultadoBusqueda:
    """Representa un resultado de búsqueda."""
    contenido: str
    score: float
    metadata: Dict


class RAGPipeline:
    """
    Pipeline RAG completo con indexación, búsqueda y generación.
    
    Esta implementación está diseñada para ser didáctica y mostrar
    claramente cada paso del proceso RAG.
    """
    
    def __init__(
        self, 
        embedding_model: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        collection_name: str = "rag_collection",
        chunk_size: int = 500,
        chunk_overlap: int = 100
    ):
        """
        Inicializa el pipeline RAG.
        
        Args:
            embedding_model: Nombre del modelo de embeddings
            collection_name: Nombre de la colección en ChromaDB
            chunk_size: Tamaño máximo de cada chunk en caracteres
            chunk_overlap: Solapamiento entre chunks
        """
        print("🔧 Inicializando RAG Pipeline...")
        
        # Modelo de embeddings
        print(f"   📐 Cargando modelo de embeddings: {embedding_model}")
        self.embedding_model = SentenceTransformer(embedding_model)
        
        # Splitter para chunking
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ". ", " "]
        )
        
        # Vector store (ChromaDB)
        print(f"   📦 Configurando ChromaDB: {collection_name}")
        self.client = chromadb.Client()
        
        # Función de embedding para ChromaDB
        self.embedding_fn = chromadb.utils.embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=embedding_model
        )
        
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            embedding_function=self.embedding_fn,
            metadata={"hnsw:space": "cosine"}
        )
        
        # Contador para IDs
        self._doc_counter = 0
        
        print("✅ Pipeline inicializado correctamente\n")
    
    def indexar_documentos(self, documentos: List[Documento]) -> int:
        """
        Indexa una lista de documentos en el vector store.
        
        Proceso:
        1. Divide cada documento en chunks
        2. Genera embeddings (automático por ChromaDB)
        3. Almacena en el vector store
        
        Returns:
            Número de chunks indexados
        """
        print(f"📥 Indexando {len(documentos)} documentos...")
        
        todos_chunks = []
        todos_metadatos = []
        todos_ids = []
        
        for doc in documentos:
            # Dividir en chunks
            chunks = self.text_splitter.split_text(doc.contenido)
            
            for i, chunk in enumerate(chunks):
                # Preparar metadatos del chunk
                chunk_metadata = doc.metadata.copy()
                chunk_metadata['chunk_index'] = i
                chunk_metadata['total_chunks'] = len(chunks)
                
                todos_chunks.append(chunk)
                todos_metadatos.append(chunk_metadata)
                todos_ids.append(f"doc_{self._doc_counter:05d}")
                self._doc_counter += 1
        
        # Añadir a ChromaDB
        if todos_chunks:
            self.collection.add(
                documents=todos_chunks,
                metadatas=todos_metadatos,
                ids=todos_ids
            )
        
        print(f"✅ {len(todos_chunks)} chunks indexados")
        return len(todos_chunks)
    
    def buscar(self, consulta: str, k: int = 3, filtros: Dict = None) -> List[ResultadoBusqueda]:
        """
        Busca los documentos más relevantes para una consulta.
        
        Args:
            consulta: Pregunta o texto de búsqueda
            k: Número de resultados a devolver
            filtros: Diccionario de filtros de metadatos (opcional)
        
        Returns:
            Lista de ResultadoBusqueda ordenados por relevancia
        """
        resultados = self.collection.query(
            query_texts=[consulta],
            n_results=k,
            where=filtros,
            include=["documents", "metadatas", "distances"]
        )
        
        # Convertir a objetos ResultadoBusqueda
        busquedas = []
        for i in range(len(resultados['documents'][0])):
            busquedas.append(ResultadoBusqueda(
                contenido=resultados['documents'][0][i],
                score=1 - resultados['distances'][0][i],  # Convertir distancia a similitud
                metadata=resultados['metadatas'][0][i]
            ))
        
        return busquedas
    
    def construir_contexto(self, resultados: List[ResultadoBusqueda]) -> str:
        """
        Construye el contexto para el LLM a partir de los resultados.
        
        Formatea los documentos recuperados de forma estructurada
        para que el LLM pueda utilizarlos efectivamente.
        """
        partes_contexto = []
        
        for i, res in enumerate(resultados, 1):
            fuente = res.metadata.get('fuente', 'Desconocida')
            pagina = res.metadata.get('pagina', 'N/A')
            
            parte = f"[Documento {i} - Fuente: {fuente}, Pág. {pagina}]\n{res.contenido}"
            partes_contexto.append(parte)
        
        return "\n\n".join(partes_contexto)
    
    def generar_prompt(self, consulta: str, contexto: str) -> str:
        """
        Genera el prompt completo para el LLM.
        
        Este prompt está diseñado para:
        - Instruir al LLM a usar solo el contexto proporcionado
        - Solicitar citaciones de las fuentes
        - Manejar casos donde no hay información suficiente
        """
        prompt = f"""Eres un asistente de investigación científica. Tu tarea es responder 
preguntas basándote ÚNICAMENTE en el contexto proporcionado.

REGLAS IMPORTANTES:
1. Usa SOLO la información del contexto para responder
2. Si la información no está en el contexto, di: "No tengo información sobre esto en los documentos disponibles"
3. Cita las fuentes usando el formato [Documento X]
4. Sé preciso y conciso en tu respuesta
5. Si hay información contradictoria, menciona ambas perspectivas

CONTEXTO:
---
{contexto}
---

PREGUNTA: {consulta}

RESPUESTA:"""
        
        return prompt
    
    def consultar(self, pregunta: str, k: int = 3, filtros: Dict = None) -> Dict:
        """
        Ejecuta el pipeline RAG completo.
        
        Esta función combina todos los pasos:
        1. Búsqueda de documentos relevantes
        2. Construcción del contexto
        3. Generación del prompt
        
        Returns:
            Diccionario con prompt, contexto y documentos fuente
        """
        # Paso 1: Buscar documentos relevantes
        resultados = self.buscar(pregunta, k=k, filtros=filtros)
        
        # Paso 2: Construir contexto
        contexto = self.construir_contexto(resultados)
        
        # Paso 3: Generar prompt
        prompt = self.generar_prompt(pregunta, contexto)
        
        return {
            'prompt': prompt,
            'contexto': contexto,
            'documentos_fuente': resultados,
            'num_documentos': len(resultados)
        }


print("✅ Clase RAGPipeline definida correctamente")

In [ ]:
# ============================================================
# Crear y usar el pipeline
# ============================================================

# Inicializar el pipeline
rag = RAGPipeline(
    embedding_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    collection_name="demo_rag",
    chunk_size=400,
    chunk_overlap=80
)

In [ ]:
# ============================================================
# Preparar e indexar documentos
# ============================================================

# Crear documentos de ejemplo
docs_ejemplo = [
    Documento(
        contenido="""Las proteínas chaperonas son moléculas esenciales para la vida celular.
        Funcionan como "guardianes" del plegamiento proteico, asistiendo a otras proteínas
        para que adopten su estructura tridimensional correcta. Sin las chaperonas, muchas
        proteínas se plegarían incorrectamente y formarían agregados tóxicos.
        
        El mecanismo de acción implica ciclos de unión y liberación dependientes de ATP.
        Las chaperonas reconocen regiones hidrofóbicas expuestas, que normalmente estarían
        ocultas en el interior de una proteína correctamente plegada.""",
        metadata={"fuente": "manual_bioquimica.pdf", "pagina": 245, "tema": "chaperonas"}
    ),
    Documento(
        contenido="""La familia Hsp70 es una de las más importantes y conservadas evolutivamente.
        Estas chaperonas reconocen segmentos hidrofóbicos de 7-8 aminoácidos en proteínas
        desplegadas. Para funcionar óptimamente, Hsp70 requiere co-chaperonas como Hsp40
        (también conocida como DnaJ), que estimulan su actividad ATPasa.
        
        El ciclo funcional de Hsp70 incluye:
        1. Unión del sustrato en estado ATP
        2. Hidrólisis de ATP estimulada por Hsp40
        3. Liberación del sustrato tras intercambio de nucleótido""",
        metadata={"fuente": "manual_bioquimica.pdf", "pagina": 248, "tema": "chaperonas"}
    ),
    Documento(
        contenido="""El mal plegamiento de proteínas está implicado en numerosas enfermedades
        neurodegenerativas. En la enfermedad de Alzheimer, los agregados de beta-amiloide
        y tau hiperfosforilada forman placas y ovillos neurofibrilares respectivamente.
        
        En Parkinson, la proteína alfa-sinucleína se agrega formando los cuerpos de Lewy.
        Las enfermedades por priones son causadas por PrPSc, una forma mal plegada de la
        proteína priónica que puede "contagiar" su conformación a proteínas normales.""",
        metadata={"fuente": "neurologia_molecular.pdf", "pagina": 89, "tema": "patologia"}
    ),
    Documento(
        contenido="""CRISPR-Cas9 ha revolucionado la edición genética. Este sistema utiliza
        un ARN guía para dirigir la nucleasa Cas9 a una secuencia específica del genoma,
        donde realiza un corte de doble hebra. La célula repara este corte mediante
        unión de extremos no homólogos (NHEJ) o recombinación homóloga (HDR).
        
        Aplicaciones incluyen: corrección de mutaciones genéticas, creación de modelos
        de enfermedad, desarrollo de terapias génicas y mejora de cultivos.""",
        metadata={"fuente": "genetica_moderna.pdf", "pagina": 156, "tema": "genetica"}
    ),
]

# Indexar documentos
num_chunks = rag.indexar_documentos(docs_ejemplo)

In [ ]:
# ============================================================
# Realizar consultas con el pipeline
# ============================================================

pregunta = "¿Cómo funcionan las proteínas chaperonas y qué enfermedades están relacionadas con su mal funcionamiento?"

print("🔍 CONSULTA RAG")
print("="*70)
print(f"Pregunta: {pregunta}\n")

# Ejecutar pipeline
resultado = rag.consultar(pregunta, k=3)

# Mostrar documentos recuperados
print("📄 DOCUMENTOS RECUPERADOS:")
print("-"*70)
for i, doc in enumerate(resultado['documentos_fuente'], 1):
    print(f"\n{i}. [Similitud: {doc.score:.4f}]")
    print(f"   Fuente: {doc.metadata.get('fuente')}, pág. {doc.metadata.get('pagina')}")
    print(f"   {doc.contenido[:100]}...")

print("\n" + "="*70)
print("📝 PROMPT GENERADO PARA EL LLM:")
print("-"*70)
print(resultado['prompt'][:1500] + "...\n")

print("💡 Este prompt se enviaría al LLM (Llama, Mistral, etc.) para generar la respuesta final.")

---

<a id="6-avanzadas"></a>
# 6. Técnicas Avanzadas de Retrieval

El RAG básico ("Naive RAG") funciona bien en muchos casos, pero existen técnicas avanzadas que pueden mejorar significativamente la calidad de las respuestas.

## 6.1 Limitaciones del RAG Básico

| Limitación | Descripción | Impacto |
|------------|-------------|----------|
| **Búsqueda solo semántica** | No captura coincidencias de términos exactos | Falla con términos técnicos específicos |
| **Sin reordenamiento** | Primer resultado puede no ser el mejor | Contexto subóptimo para el LLM |
| **Query única** | La consulta del usuario puede ser ambigua | Baja cobertura de documentos relevantes |
| **Sin verificación** | El LLM puede ignorar el contexto | Posibles alucinaciones |

## 6.2 Hybrid Search: Combinando Semántica y Keywords

### El Problema

La búsqueda puramente semántica puede fallar con:
- **Términos técnicos**: "Hsp70" vs "proteína de choque térmico 70"
- **Acrónimos**: "PCR", "ADN", "ARNm"
- **Nombres propios**: "CRISPR-Cas9", "Alzheimer"

### La Solución: Búsqueda Híbrida

Combina dos tipos de búsqueda:

1. **Búsqueda Vectorial (Semántica)**: Captura el significado
2. **Búsqueda Léxica (BM25)**: Captura coincidencias de términos exactos

```
Score_final = α × Score_vectorial + (1-α) × Score_BM25

donde α ∈ [0, 1] controla el balance:
  α = 1.0 → Solo vectorial
  α = 0.0 → Solo BM25
  α = 0.5 → Balance igual (recomendado como punto de partida)
```

In [ ]:
# ============================================================
# Implementación de Búsqueda Híbrida
# ============================================================

from rank_bm25 import BM25Okapi
import re
import numpy as np
from typing import List, Tuple

class BusquedaHibrida:
    """
    Implementa búsqueda híbrida combinando vectorial + BM25.
    
    BM25 (Best Matching 25) es un algoritmo de ranking que considera:
    - Frecuencia del término en el documento (TF)
    - Frecuencia inversa del documento (IDF)
    - Longitud del documento
    """
    
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model
        self.documentos = []
        self.embeddings = None
        self.bm25 = None
        self.tokenized_docs = []
    
    def _tokenizar(self, texto: str) -> List[str]:
        """Tokenización simple para BM25."""
        # Convertir a minúsculas y extraer palabras
        tokens = re.findall(r'\w+', texto.lower())
        return tokens
    
    def indexar(self, documentos: List[str]):
        """Indexa documentos para búsqueda híbrida."""
        self.documentos = documentos
        
        # Índice vectorial
        print("   Generando embeddings...")
        self.embeddings = self.embedding_model.encode(documentos)
        
        # Índice BM25
        print("   Construyendo índice BM25...")
        self.tokenized_docs = [self._tokenizar(doc) for doc in documentos]
        self.bm25 = BM25Okapi(self.tokenized_docs)
        
        print(f"✅ {len(documentos)} documentos indexados para búsqueda híbrida")
    
    def buscar(self, query: str, k: int = 5, alpha: float = 0.5) -> List[Tuple[str, float, dict]]:
        """
        Realiza búsqueda híbrida.
        
        Args:
            query: Consulta del usuario
            k: Número de resultados
            alpha: Peso de búsqueda vectorial (0-1)
                   1.0 = solo vectorial, 0.0 = solo BM25
        
        Returns:
            Lista de (documento, score_hibrido, detalles)
        """
        # Búsqueda vectorial
        query_embedding = self.embedding_model.encode([query])
        vector_scores = cosine_similarity(query_embedding, self.embeddings)[0]
        
        # Búsqueda BM25
        tokenized_query = self._tokenizar(query)
        bm25_scores = self.bm25.get_scores(tokenized_query)
        
        # Normalizar scores a rango [0, 1]
        vector_norm = self._normalizar(vector_scores)
        bm25_norm = self._normalizar(bm25_scores)
        
        # Combinar scores
        hybrid_scores = alpha * vector_norm + (1 - alpha) * bm25_norm
        
        # Obtener top-k
        top_indices = np.argsort(hybrid_scores)[::-1][:k]
        
        resultados = []
        for idx in top_indices:
            resultados.append((
                self.documentos[idx],
                hybrid_scores[idx],
                {
                    'score_vectorial': vector_scores[idx],
                    'score_bm25': bm25_scores[idx],
                    'score_vectorial_norm': vector_norm[idx],
                    'score_bm25_norm': bm25_norm[idx]
                }
            ))
        
        return resultados
    
    def _normalizar(self, scores):
        """Normaliza scores al rango [0, 1]."""
        min_s, max_s = scores.min(), scores.max()
        if max_s - min_s < 1e-6:
            return np.zeros_like(scores)
        return (scores - min_s) / (max_s - min_s)


# Crear instancia
print("🔧 Inicializando búsqueda híbrida...")
hibrida = BusquedaHibrida(embedding_model)

# Indexar documentos de ejemplo
docs_hibrida = [
    "Las proteínas chaperonas Hsp70 y Hsp90 son esenciales para el plegamiento proteico.",
    "La técnica CRISPR-Cas9 permite edición genética precisa del ADN.",
    "El mal plegamiento de proteínas causa enfermedades como Alzheimer.",
    "Hsp70 reconoce secuencias hidrofóbicas de 7-8 aminoácidos.",
    "La PCR amplifica secuencias específicas de ADN usando Taq polimerasa.",
    "Los agregados de alfa-sinucleína causan la enfermedad de Parkinson.",
]

hibrida.indexar(docs_hibrida)

In [ ]:
# ============================================================
# Comparación: Vectorial vs BM25 vs Híbrido
# ============================================================

# Query con término técnico específico
query = "Hsp70 aminoácidos hidrofóbicos"

print(f"🔍 COMPARACIÓN DE MÉTODOS DE BÚSQUEDA")
print(f"="*70)
print(f"Query: \"{query}\"\n")

# Solo vectorial (alpha=1.0)
print("1️⃣ SOLO VECTORIAL (alpha=1.0):")
res_vec = hibrida.buscar(query, k=3, alpha=1.0)
for i, (doc, score, _) in enumerate(res_vec, 1):
    print(f"   {i}. [{score:.3f}] {doc[:60]}...")

# Solo BM25 (alpha=0.0)
print("\n2️⃣ SOLO BM25 (alpha=0.0):")
res_bm25 = hibrida.buscar(query, k=3, alpha=0.0)
for i, (doc, score, _) in enumerate(res_bm25, 1):
    print(f"   {i}. [{score:.3f}] {doc[:60]}...")

# Híbrido (alpha=0.5)
print("\n3️⃣ HÍBRIDO (alpha=0.5):")
res_hyb = hibrida.buscar(query, k=3, alpha=0.5)
for i, (doc, score, detalles) in enumerate(res_hyb, 1):
    print(f"   {i}. [{score:.3f}] (vec:{detalles['score_vectorial_norm']:.2f}, bm25:{detalles['score_bm25_norm']:.2f})")
    print(f"      {doc[:55]}...")

print("\n💡 Observa cómo BM25 encuentra mejor el documento con 'Hsp70' y 'aminoácidos',")
print("   mientras que vectorial captura el significado general.")

## 6.3 Reranking: Mejorando la Precisión

### El Concepto

El **reranking** es un paso adicional que reordena los resultados iniciales usando un modelo más preciso pero más lento.

```
Query → Retrieval (rápido, top-20) → Reranker (preciso, top-5) → LLM
```

### Bi-Encoder vs Cross-Encoder

| Aspecto | Bi-Encoder | Cross-Encoder |
|---------|------------|---------------|
| **Cómo funciona** | Codifica query y doc por separado | Procesa query + doc juntos |
| **Velocidad** | Muy rápido (O(1) por comparación) | Lento (procesa cada par) |
| **Precisión** | Buena | Excelente |
| **Uso** | Retrieval inicial | Reranking |

### Flujo Típico

1. **Retrieval**: Bi-encoder recupera top-20 candidatos (rápido)
2. **Reranking**: Cross-encoder reordena → top-5 (preciso)
3. **Generación**: LLM usa los 5 mejores documentos

In [ ]:
# ============================================================
# Implementación de Reranking con Cross-Encoder
# ============================================================

from sentence_transformers import CrossEncoder

# Cargar modelo de reranking
print("🔧 Cargando modelo de reranking (Cross-Encoder)...")
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("✅ Modelo de reranking cargado\n")


def rerank(query: str, documentos: List[str], top_k: int = 3) -> List[Tuple[str, float]]:
    """
    Reordena documentos usando Cross-Encoder.
    
    El Cross-Encoder evalúa directamente la relevancia de cada par
    (query, documento), lo que es más preciso que comparar embeddings.
    """
    # Crear pares (query, documento)
    pares = [[query, doc] for doc in documentos]
    
    # Obtener scores del reranker
    scores = reranker.predict(pares)
    
    # Ordenar por score descendente
    doc_scores = list(zip(documentos, scores))
    doc_scores.sort(key=lambda x: x[1], reverse=True)
    
    return doc_scores[:top_k]


# Ejemplo de reranking
query = "enfermedades causadas por proteínas que no se pliegan bien"

print(f"🔍 DEMOSTRACIÓN DE RERANKING")
print(f"="*70)
print(f"Query: \"{query}\"\n")

# Orden original (simulando retrieval inicial)
print("📄 ORDEN ORIGINAL (del retrieval):")
for i, doc in enumerate(docs_hibrida, 1):
    print(f"   {i}. {doc[:60]}...")

# Después de reranking
print("\n📄 DESPUÉS DE RERANKING:")
reranked = rerank(query, docs_hibrida, top_k=3)
for i, (doc, score) in enumerate(reranked, 1):
    print(f"   {i}. [Score: {score:.4f}] {doc[:50]}...")

print("\n💡 El reranker identifica correctamente los documentos sobre")
print("   enfermedades y mal plegamiento como los más relevantes.")

## 6.4 Multi-Query RAG: Expandiendo la Búsqueda

### El Problema

Las consultas de los usuarios pueden ser:
- **Ambiguas**: "¿Cómo funcionan las proteínas?" (muy general)
- **Incompletas**: No usan todos los términos relevantes
- **Mal formuladas**: No coinciden con cómo está escrita la información

### La Solución: Múltiples Consultas

Generar variaciones de la consulta original y fusionar los resultados:

```
Query original: "¿Qué hacen las chaperonas?"
                        │
                        ▼
        ┌───────────────┼───────────────┐
        ▼               ▼               ▼
   "función de      "cómo ayudan     "proteínas Hsp70
    chaperonas"      las chaperonas"   Hsp90 función"
        │               │               │
        ▼               ▼               ▼
    [Retrieval]    [Retrieval]    [Retrieval]
        │               │               │
        └───────────────┼───────────────┘
                        ▼
                   [Fusión RRF]
                        │
                        ▼
              Resultados combinados
```

### Reciprocal Rank Fusion (RRF)

Técnica para combinar rankings de múltiples fuentes:

$$RRF(d) = \sum_{q \in Q} \frac{1}{k + rank_q(d)}$$

Donde:
- $d$ = documento
- $Q$ = conjunto de queries
- $rank_q(d)$ = posición del documento $d$ en los resultados de query $q$
- $k$ = constante (típicamente 60)

In [ ]:
# ============================================================
# Implementación de Multi-Query RAG
# ============================================================

def generar_queries_alternativas(query_original: str) -> List[str]:
    """
    Genera variaciones de la consulta original.
    
    En producción, usarías un LLM para generar estas variaciones.
    Aquí simulamos algunas transformaciones comunes.
    """
    variaciones = [query_original]  # Siempre incluir la original
    
    # Ejemplo de variaciones manuales (en producción, usar LLM)
    if "chaperona" in query_original.lower():
        variaciones.extend([
            "proteínas de choque térmico HSP función",
            "Hsp70 Hsp90 plegamiento proteico",
            "cómo asisten las chaperonas moleculares"
        ])
    elif "enfermedad" in query_original.lower():
        variaciones.extend([
            "patologías proteínas mal plegadas",
            "Alzheimer Parkinson agregados proteicos"
        ])
    
    return variaciones


def reciprocal_rank_fusion(rankings: List[List[str]], k: int = 60) -> List[Tuple[str, float]]:
    """
    Fusiona múltiples rankings usando Reciprocal Rank Fusion.
    
    Args:
        rankings: Lista de rankings (cada uno es una lista de documentos ordenados)
        k: Constante de suavizado (default: 60)
    
    Returns:
        Lista de (documento, score_rrf) ordenada por score
    """
    rrf_scores = {}
    
    for ranking in rankings:
        for rank, doc in enumerate(ranking, 1):
            if doc not in rrf_scores:
                rrf_scores[doc] = 0
            rrf_scores[doc] += 1 / (k + rank)
    
    # Ordenar por score RRF
    sorted_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_docs


# Demostración
query_original = "¿Qué hacen las chaperonas moleculares?"

print(f"🔍 MULTI-QUERY RAG")
print(f"="*70)
print(f"Query original: \"{query_original}\"\n")

# Generar queries alternativas
queries = generar_queries_alternativas(query_original)
print("📝 Queries generadas:")
for i, q in enumerate(queries, 1):
    print(f"   {i}. {q}")

# Realizar búsquedas con cada query
print("\n🔍 Resultados por query:")
rankings = []
for q in queries:
    resultados = hibrida.buscar(q, k=4, alpha=0.5)
    ranking = [doc for doc, _, _ in resultados]
    rankings.append(ranking)

# Fusionar con RRF
fusion = reciprocal_rank_fusion(rankings)

print("\n📄 RESULTADOS FUSIONADOS (RRF):")
for i, (doc, score) in enumerate(fusion[:4], 1):
    print(f"   {i}. [RRF: {score:.4f}] {doc[:55]}...")

## 6.5 Resumen de Técnicas Avanzadas

| Técnica | Cuándo Usar | Mejora |
|---------|-------------|--------|
| **Hybrid Search** | Términos técnicos, acrónimos, nombres propios | Mejor recall para términos específicos |
| **Reranking** | Alta precisión necesaria, tienes recursos computacionales | Mejor precisión en top-k |
| **Multi-Query** | Consultas ambiguas, baja cobertura | Mayor cobertura de documentos relevantes |
| **Query Expansion** | Vocabulario variado en documentos | Mejor matching de términos |

### Pipeline Avanzado Recomendado

```
Query → [Multi-Query] → [Hybrid Search] → [Reranking] → [LLM]
         (expandir)      (recuperar)       (precisar)   (generar)
```

---

<a id="7-evaluacion"></a>
# 7. Evaluación y Métricas de RAG

## 7.1 ¿Por qué Evaluar?

Sin evaluación sistemática, no puedes:
- Saber si tu sistema RAG está funcionando bien
- Comparar diferentes configuraciones
- Detectar regresiones al hacer cambios
- Justificar mejoras ante stakeholders

## 7.2 Métricas de Retrieval

Evalúan la calidad de los documentos recuperados:

### Precision@k

¿Qué proporción de los top-k documentos recuperados son relevantes?

$$Precision@k = \frac{|\text{documentos relevantes en top-k}|}{k}$$

**Ejemplo**: Si de los top-5 resultados, 3 son relevantes → Precision@5 = 0.6

### Recall@k

¿Qué proporción de todos los documentos relevantes fueron recuperados?

$$Recall@k = \frac{|\text{documentos relevantes en top-k}|}{|\text{total documentos relevantes}|}$$

**Ejemplo**: Si hay 10 docs relevantes y recuperamos 3 en top-5 → Recall@5 = 0.3

### MRR (Mean Reciprocal Rank)

¿Qué tan arriba aparece el primer documento relevante?

$$MRR = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{rank_i}$$

**Ejemplo**: Si el primer doc relevante está en posición 2 → RR = 0.5

### Hit Rate (Success Rate)

¿Hay al menos un documento relevante en los top-k?

$$HitRate@k = \frac{|\text{queries con al menos 1 doc relevante en top-k}|}{|\text{total queries}|}$$

## 7.3 Métricas de Generación

Evalúan la calidad de la respuesta del LLM:

### Faithfulness (Fidelidad)

¿La respuesta se basa en el contexto proporcionado?
- Detecta alucinaciones
- Verifica que no invente información

### Answer Relevancy

¿La respuesta contesta la pregunta del usuario?
- No basta con ser fiel al contexto
- Debe ser pertinente a la consulta

### Context Precision

¿Los documentos del contexto eran realmente necesarios?
- Penaliza incluir información irrelevante

In [ ]:
# ============================================================
# Implementación de Métricas de Retrieval
# ============================================================

def precision_at_k(retrieved: List[str], relevant: set, k: int) -> float:
    """
    Calcula Precision@k.
    
    Args:
        retrieved: Lista de documentos recuperados (ordenados por ranking)
        relevant: Conjunto de documentos relevantes (ground truth)
        k: Número de documentos a considerar
    """
    retrieved_k = retrieved[:k]
    relevant_in_k = sum(1 for doc in retrieved_k if doc in relevant)
    return relevant_in_k / k


def recall_at_k(retrieved: List[str], relevant: set, k: int) -> float:
    """Calcula Recall@k."""
    if not relevant:
        return 0.0
    retrieved_k = retrieved[:k]
    relevant_in_k = sum(1 for doc in retrieved_k if doc in relevant)
    return relevant_in_k / len(relevant)


def mrr(retrieved: List[str], relevant: set) -> float:
    """Calcula Mean Reciprocal Rank (para una query)."""
    for rank, doc in enumerate(retrieved, 1):
        if doc in relevant:
            return 1 / rank
    return 0.0


def hit_rate_at_k(retrieved: List[str], relevant: set, k: int) -> float:
    """Calcula Hit Rate@k (1 si hay al menos un relevante, 0 si no)."""
    retrieved_k = retrieved[:k]
    return 1.0 if any(doc in relevant for doc in retrieved_k) else 0.0


# Ejemplo de evaluación
print("📊 EVALUACIÓN DE RETRIEVAL")
print("="*70)

# Simulamos resultados de retrieval
retrieved_docs = ["doc_A", "doc_B", "doc_C", "doc_D", "doc_E"]
relevant_docs = {"doc_A", "doc_C", "doc_F", "doc_G"}  # Ground truth

print(f"\nDocumentos recuperados: {retrieved_docs}")
print(f"Documentos relevantes (ground truth): {relevant_docs}")
print()

for k in [1, 3, 5]:
    p = precision_at_k(retrieved_docs, relevant_docs, k)
    r = recall_at_k(retrieved_docs, relevant_docs, k)
    hr = hit_rate_at_k(retrieved_docs, relevant_docs, k)
    print(f"k={k}: Precision={p:.2f}, Recall={r:.2f}, Hit Rate={hr:.2f}")

print(f"\nMRR: {mrr(retrieved_docs, relevant_docs):.2f}")
print("   (El primer doc relevante 'doc_A' está en posición 1 → MRR = 1.0)")

---

<a id="8-prompting"></a>
# 8. Prompt Engineering para RAG

## 8.1 La Importancia del Prompt

El prompt es la interfaz entre el contexto recuperado y la respuesta generada. Un buen prompt debe:

1. **Instruir claramente** al modelo sobre su rol y tarea
2. **Estructurar el contexto** de forma que sea fácil de procesar
3. **Guiar el formato** de la respuesta esperada
4. **Manejar edge cases** (información insuficiente, contradicciones)

## 8.2 Estructura de un Prompt RAG Efectivo

```
┌─────────────────────────────────────────────────────────────┐
│ SYSTEM PROMPT                                               │
│ - Rol del asistente                                        │
│ - Instrucciones de comportamiento                          │
│ - Reglas para usar el contexto                             │
│ - Formato de respuesta esperado                            │
├─────────────────────────────────────────────────────────────┤
│ CONTEXTO                                                    │
│ - Documentos recuperados                                   │
│ - Metadatos (fuente, página, fecha)                        │
│ - Delimitadores claros entre documentos                    │
├─────────────────────────────────────────────────────────────┤
│ PREGUNTA                                                    │
│ - La consulta del usuario                                  │
├─────────────────────────────────────────────────────────────┤
│ INSTRUCCIONES DE RESPUESTA                                  │
│ - Cómo estructurar la respuesta                            │
│ - Requisitos de citación                                   │
│ - Qué hacer si no hay información                          │
└─────────────────────────────────────────────────────────────┘
```

## 8.3 Plantillas de Prompt por Caso de Uso

In [ ]:
# ============================================================
# Plantillas de Prompts para RAG
# ============================================================

PROMPTS_RAG = {
    # Prompt para investigación científica
    "cientifico": '''Eres un asistente de investigación científica especializado en {dominio}.

INSTRUCCIONES:
1. Responde ÚNICAMENTE basándote en el contexto proporcionado
2. Si la información no está en el contexto, di claramente: "No tengo información sobre esto en los documentos disponibles"
3. Cita siempre las fuentes usando el formato [Documento X, pág. Y]
4. Usa terminología científica precisa
5. Si hay información contradictoria, menciona ambas perspectivas
6. Distingue entre hechos establecidos y hipótesis

CONTEXTO:
---
{contexto}
---

PREGUNTA: {pregunta}

RESPUESTA:''',

    # Prompt para documentación técnica
    "tecnico": '''Eres un asistente técnico que ayuda con documentación de sistemas y procedimientos.

FORMATO DE RESPUESTA:
1. Resumen breve (1-2 oraciones)
2. Pasos detallados si es un procedimiento
3. Advertencias o notas importantes
4. Referencias a documentos específicos

REGLAS:
- Usa solo la información del contexto
- Sé preciso con comandos, configuraciones y parámetros
- Indica la versión del sistema si está disponible

DOCUMENTACIÓN DISPONIBLE:
---
{contexto}
---

CONSULTA: {pregunta}

RESPUESTA:''',

    # Prompt con output estructurado (JSON)
    "estructurado": '''Analiza el contexto y responde la pregunta en formato JSON.

CONTEXTO:
---
{contexto}
---

PREGUNTA: {pregunta}

Responde con el siguiente formato JSON:
{{
    "respuesta": "Tu respuesta basada en el contexto",
    "confianza": "alta|media|baja",
    "fuentes_usadas": ["lista de fuentes citadas"],
    "informacion_faltante": "qué información adicional sería útil (o null si la respuesta es completa)"
}}

JSON:''',

    # Prompt simple para casos básicos
    "simple": '''Contexto:
{contexto}

Pregunta: {pregunta}

Responde basándote solo en el contexto. Si no hay información suficiente, dilo claramente.

Respuesta:'''
}

print("📝 PLANTILLAS DE PROMPTS DISPONIBLES:")
print("="*50)
for nombre in PROMPTS_RAG.keys():
    print(f"   ✅ {nombre}")

## 8.4 Técnicas Avanzadas de Prompting

### Chain of Thought (CoT)

Instruir al modelo a razonar paso a paso:

```
"Antes de responder, analiza el contexto paso a paso:
1. Identifica los conceptos clave en la pregunta
2. Localiza información relevante en cada documento
3. Sintetiza la información encontrada
4. Formula una respuesta coherente"
```

### Few-Shot Learning

Proporcionar ejemplos de respuestas deseadas:

```
"Ejemplos de respuestas correctas:

Pregunta: ¿Qué es X?
Respuesta: X es... [Documento 1, pág. 5]

Pregunta: ¿Cómo funciona Y?
Respuesta: Y funciona mediante... [Documento 2, pág. 12]"
```

### Self-Consistency

Generar múltiples respuestas y seleccionar la más consistente.

### Guardrails

Validar que la respuesta cumpla criterios específicos:
- ¿Cita fuentes?
- ¿Responde la pregunta?
- ¿Es coherente con el contexto?

---

<a id="9-opensource"></a>
# 9. Soluciones Open Source para RAG

## 9.1 Frameworks de Desarrollo

### LangChain

El framework más popular para construir aplicaciones con LLMs.

| Aspecto | Detalle |
|---------|----------|
| **Licencia** | MIT |
| **Lenguajes** | Python, JavaScript |
| **Integraciones** | 100+ (LLMs, vectorstores, tools) |
| **Documentación** | Excelente |
| **Comunidad** | Muy activa |
| **Ideal para** | Prototipos rápidos, producción |

### LlamaIndex

Especializado en indexación y recuperación de datos.

| Aspecto | Detalle |
|---------|----------|
| **Licencia** | MIT |
| **Fortaleza** | Manejo de documentos complejos |
| **Características** | Índices avanzados, soporte PDF/tablas |
| **Ideal para** | Repositorios documentales, papers científicos |

### Haystack

Framework europeo con enfoque en pipelines modulares.

| Aspecto | Detalle |
|---------|----------|
| **Licencia** | Apache 2.0 |
| **Origen** | Alemania (deepset) |
| **Fortaleza** | Pipelines, sensibilidad RGPD |
| **Ideal para** | Enterprise, requisitos de auditoría |

## 9.2 Aplicaciones Listas para Usar

### Open WebUI

Interfaz web similar a ChatGPT, self-hosted.

```bash
# Despliegue rápido con Docker
docker run -d -p 3000:8080 -v open-webui:/app/backend/data \
  -e OLLAMA_BASE_URL=http://host.docker.internal:11434 \
  ghcr.io/open-webui/open-webui:main
```

**Características:**
- RAG integrado (subida de documentos)
- Gestión de usuarios y permisos
- Múltiples modelos
- Interfaz intuitiva

### PrivateGPT

100% offline, máxima privacidad.

**Características:**
- Sin conexión a internet requerida
- API compatible con OpenAI
- Ideal para datos confidenciales

### AnythingLLM

Enfoque empresarial con multi-tenancy.

**Características:**
- Espacios de trabajo separados
- Múltiples proveedores de LLM
- Gestión de permisos granular

## 9.3 Modelos LLM Open Source

| Modelo | Tamaños | Licencia | Fortalezas |
|--------|---------|----------|------------|
| **Llama 3.1** | 8B, 70B, 405B | Llama 3 | Propósito general, multilingüe |
| **Mistral/Mixtral** | 7B, 8x7B, 8x22B | Apache 2.0 | Eficiencia, razonamiento |
| **Qwen 2.5** | 0.5B-72B | Apache 2.0 | Multilingüe, código |
| **Gemma 2** | 2B, 9B, 27B | Gemma | Ligero, edge computing |

## 9.4 Servidores de Inferencia

### Ollama

La forma más simple de ejecutar LLMs localmente.

```bash
# Instalación
curl -fsSL https://ollama.com/install.sh | sh

# Descargar y ejecutar un modelo
ollama pull llama3.1:8b
ollama run llama3.1:8b
```

### vLLM

Alto rendimiento para producción.

```bash
# Servidor de inferencia
python -m vllm.entrypoints.openai.api_server \
  --model meta-llama/Llama-3.1-8B-Instruct
```

---

<a id="10-produccion"></a>
# 10. Despliegue en Producción

## 10.1 Checklist de Producción

### Preparación de Datos

- [ ] Chunking optimizado para el dominio
- [ ] Metadatos enriquecidos (fuente, fecha, autor)
- [ ] Limpieza de texto (OCR, formatos)
- [ ] Pipeline de actualización definido

### Retrieval

- [ ] Hybrid search implementado si hay términos técnicos
- [ ] Reranking para alta precisión
- [ ] Filtros de metadatos configurados
- [ ] Número óptimo de documentos (k) validado

### Generación

- [ ] Prompts probados y optimizados
- [ ] Manejo de "no sé" implementado
- [ ] Citaciones funcionando correctamente
- [ ] Guardrails contra alucinaciones

### Infraestructura

- [ ] Autenticación (LDAP/SSO)
- [ ] Cifrado (TLS en tránsito, en reposo)
- [ ] Logs de auditoría
- [ ] Backups de vectorstore
- [ ] Monitorización de métricas

### Evaluación

- [ ] Conjunto de test creado
- [ ] Métricas baseline establecidas
- [ ] Sistema de feedback de usuarios
- [ ] A/B testing configurado

## 10.2 Arquitectura Recomendada

### Stack Básico (Para Empezar)

```
┌─────────────────────────────────────────────────────────┐
│                    STACK BÁSICO                         │
├─────────────────────────────────────────────────────────┤
│ LLM:        Llama 3.1 8B o Qwen 2.5 7B (via Ollama)    │
│ Embeddings: paraphrase-multilingual-MiniLM o BGE-M3    │
│ VectorDB:   ChromaDB o Qdrant                          │
│ Framework:  LangChain o LlamaIndex                     │
│ Interfaz:   Open WebUI o Gradio                        │
│ Hardware:   1x GPU 16GB (RTX 4090, A4000)              │
└─────────────────────────────────────────────────────────┘
```

### Stack Avanzado (Escala)

```
┌─────────────────────────────────────────────────────────┐
│                   STACK AVANZADO                        │
├─────────────────────────────────────────────────────────┤
│ LLM:         Llama 3.1 70B o Mixtral 8x22B             │
│ Servidor:    vLLM con balanceo de carga                │
│ VectorDB:    Qdrant cluster o Milvus                   │
│ Orquestación: Kubernetes + Helm                        │
│ Caché:       Redis para respuestas frecuentes          │
│ Monitorización: Prometheus + Grafana                    │
│ Hardware:    Múltiples GPUs A100/H100                  │
└─────────────────────────────────────────────────────────┘
```

## 10.3 Próximos Pasos

1. **Prueba de Concepto**
   - Instalar Ollama + Open WebUI
   - Probar con un conjunto pequeño de documentos
   - Validar con usuarios piloto

2. **Iteración**
   - Crear conjunto de evaluación
   - Optimizar chunking y prompts
   - Implementar técnicas avanzadas según necesidad

3. **Producción**
   - Implementar seguridad y autenticación
   - Configurar monitorización
   - Establecer procesos de actualización

---

# 🎯 Conclusión y Recursos

## Resumen de lo Aprendido

En esta guía hemos cubierto:

1. **Fundamentos**: Qué es RAG, por qué es necesario, arquitectura
2. **Embeddings**: Representación semántica, modelos, métricas
3. **Chunking**: Estrategias, parámetros óptimos, metadatos
4. **Vector Stores**: ChromaDB, FAISS, búsqueda y filtros
5. **Pipeline RAG**: Construcción completa paso a paso
6. **Técnicas Avanzadas**: Hybrid search, reranking, multi-query
7. **Evaluación**: Métricas de retrieval y generación
8. **Prompting**: Plantillas y técnicas efectivas
9. **Soluciones Open Source**: Frameworks y aplicaciones
10. **Producción**: Checklist y arquitecturas recomendadas

## Recursos Adicionales

### Documentación Oficial
- LangChain: https://langchain.com
- LlamaIndex: https://llamaindex.ai
- ChromaDB: https://www.trychroma.com
- Ollama: https://ollama.ai

### Papers Fundamentales
- "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks" (Lewis et al., 2020)
- "A Survey on Retrieval-Augmented Text Generation" (Gao et al., 2023)

### Modelos en Hugging Face
- https://huggingface.co/models

---

